<a href="https://colab.research.google.com/github/Jvcklyn/snu_hort/blob/main/Final_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/gdrive') #Google Drive로 경로 지정

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate #교차검증 K-Fold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score #정확도, 재현율, 정밀도
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [54]:
total=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset/홍삼액기스 국가별 ginsenoside 분석표(select)_normalized.csv')
result=pd.DataFrame.to_numpy(total)
df_result=pd.DataFrame(result)
target=df_result[0]
data=df_result.iloc[:,1:]

x_train, x_test, y_train, y_test=train_test_split(data, target, test_size=0.2, shuffle=True) #random_state=n
print(df_result)

total2=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset/홍삼액기스 국가별 ginsenoside 분석표_normalized(수정).csv')
result2=pd.DataFrame.to_numpy(total2)
df_result2=pd.DataFrame(result2)
target2=df_result2[0]
data2=df_result2.iloc[:,1:]
x_train2, x_test2, y_train2, y_test2=train_test_split(data2, target2, test_size=0.2, shuffle=True) #random_state=n

print(df_result2)

       0          1          2   ...        9         10        11
0   Korea  -0.286736   -0.31255  ...  0.169449 -0.136083 -0.151448
1   Korea  -0.683101  -0.534781  ...  0.175124 -0.136083 -0.151448
2   Korea  -0.516002  -0.175945  ...  0.345654 -0.136083 -0.151448
3   Korea   0.225749 -0.0360187  ...  0.794594 -0.136083 -0.151448
4   Korea   0.317714   0.212664  ...  0.346515 -0.136083 -0.151448
..    ...        ...        ...  ...       ...       ...       ...
95  China  -0.916186  -0.285496  ...   -1.3047 -0.377916 -0.727621
96  China -0.0286783  -0.285496  ...  0.729364 -0.377916  0.238744
97  China   0.566928  -0.285496  ... -0.153572 -0.377916   2.28231
98  China     2.0431  -0.285496  ...   2.11974 -0.377916   5.63062
99  China  -0.789333  -0.285496  ...  -1.35427 -0.377916 -0.736708

[100 rows x 12 columns]
       0          1          2   ...        21        22        23
0   Korea  -0.286736   -0.31255  ... -0.151448 -0.220021         0
1   Korea  -0.683101  -0.534781  ... 

In [55]:
#의사결정나무 분석법
print("Decision Tree")

clf=DecisionTreeClassifier(random_state=5)
clf.fit(x_train, y_train)
clf_pred=clf.predict(x_test)
clf_acc=accuracy_score(y_test, clf_pred)
clf_rec=recall_score(y_test, clf_pred, average='micro')
clf_pre=precision_score(y_test, clf_pred, average='micro')

clf2=DecisionTreeClassifier(random_state=41)
clf2.fit(x_train2, y_train2)
clf_pred2=clf2.predict(x_test2)
clf_acc2=accuracy_score(y_test2, clf_pred2)
clf_rec2=recall_score(y_test2, clf_pred2, average='micro')
clf_pre2=precision_score(y_test2, clf_pred2, average='micro')

clf_kscore=(cross_val_score(clf, data, target, cv=5))
clf_kscore2=(cross_val_score(clf2, data, target, cv=5))

print("1차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(clf_acc, clf_rec, clf_pre))
print("2차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(clf_acc2, clf_rec2, clf_pre2))
print("K-Fold cross validation score: %.4f, %.4f"%(clf_kscore.mean(), clf_kscore2.mean()))

Decision Tree
1차 데이터 예측 정확도: 0.9500, 0.9500, 0.9500
2차 데이터 예측 정확도: 0.9500, 0.9500, 0.9500
K-Fold cross validation score: 0.9700, 0.9700


In [56]:
#KNN 분석법
print("K-Nearest-Neighbors")

knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
knn2=KNeighborsClassifier(n_neighbors=5)
knn2.fit(x_train2, y_train2)

y_knn_pred=knn.predict(x_test)
knn_acc=accuracy_score(y_test, y_knn_pred)
knn_rec=recall_score(y_test, y_knn_pred, average='micro')
knn_pre=precision_score(y_test, y_knn_pred, average='micro')

y_knn_pred2=knn2.predict(x_test2)
knn_acc2=accuracy_score(y_test2, y_knn_pred2)
knn_rec2=recall_score(y_test, y_knn_pred2, average='micro')
knn_pre2=precision_score(y_test, y_knn_pred2, average='micro')

print("1차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(knn_acc, knn_rec, knn_pre))
print("2차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(knn_acc2, knn_rec2, knn_pre2))

knn_kscore=(cross_val_score(knn, data, target, cv=5))
knn_kscore2=(cross_val_score(knn2, data, target, cv=5))
print("K-Fold cross validation score: %.4f, %.4f"%(knn_kscore.mean(), knn_kscore2.mean()))

K-Nearest-Neighbors
1차 데이터 예측 정확도: 0.8500, 0.8500, 0.8500
2차 데이터 예측 정확도: 0.8500, 0.6000, 0.6000
K-Fold cross validation score: 0.7600, 0.7600


In [57]:
#Linear Discriminant Analysis
print("Linear Discriminant Analysis")

lda=LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)
lda_pred=lda.predict(x_test)
lda_acc=accuracy_score(y_test, lda_pred)
lda_rec=recall_score(y_test, lda_pred, average='micro')
lda_pre=precision_score(y_test, lda_pred, average='micro')

lda2=LinearDiscriminantAnalysis()
lda2.fit(x_train2, y_train2)
lda2_pred=lda2.predict(x_test2)
lda_acc2=accuracy_score(y_test2, lda2_pred)
lda_rec2=recall_score(y_test, lda2_pred, average='micro')
lda_pre2=precision_score(y_test, lda2_pred, average='micro')

print("1차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(lda_acc, lda_rec, lda_pre))
print("2차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(lda_acc2, lda_rec2, lda_pre2))

lda_kscore=(cross_val_score(lda, data, target, cv=5))
lda_kscore2=(cross_val_score(lda2, data, target, cv=5))
print("K-Fold cross validation score: %.4f, %.4f"%(lda_kscore.mean(), lda_kscore2.mean()))

Linear Discriminant Analysis
1차 데이터 예측 정확도: 0.3000, 0.3000, 0.3000
2차 데이터 예측 정확도: 0.4000, 0.4500, 0.4500
K-Fold cross validation score: 0.3500, 0.3500


In [58]:
#Support Vector Machine
print("Support Vector Machine")

svc=SVC(kernel='rbf')
svc.fit(x_train, y_train)

svc2=SVC(kernel='rbf')
svc2.fit(x_train2, y_train2)

y_svc_pred=svc.predict(x_test)
svc_acc=accuracy_score(y_test, y_svc_pred)
svc_rec=recall_score(y_test, y_svc_pred, average='micro')
svc_pre=precision_score(y_test, y_svc_pred, average='micro')

y_svc_pred2=svc2.predict(x_test2)
svc_acc2=accuracy_score(y_test2, y_svc_pred2)
svc_rec2=recall_score(y_test, y_svc_pred2, average='micro')
svc_pre2=precision_score(y_test, y_svc_pred2, average='micro')

print("1차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(svc_acc, svc_rec, svc_pre))
print("2차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(svc_acc2, svc_rec2, svc_pre2))

svc_kscore=(cross_val_score(svc, data, target, cv=5))
svc_kscore2=(cross_val_score(svc2, data, target, cv=5))
print("K-Fold cross validation score: %.4f, %.4f"%(svc_kscore.mean(), svc_kscore2.mean()))

Support Vector Machine
1차 데이터 예측 정확도: 0.9500, 0.9500, 0.9500
2차 데이터 예측 정확도: 0.7500, 0.7000, 0.7000
K-Fold cross validation score: 0.8000, 0.8000


In [59]:
#Logistic Regression
print("Logistic Regression")

lrc=LogisticRegression()
lrc.fit(x_train, y_train)

lrc2=LogisticRegression()
lrc2.fit(x_train2, y_train2)

y_lrc_pred=lrc.predict(x_test)
lrc_acc=accuracy_score(y_test, y_lrc_pred) 
lrc_rec=recall_score(y_test, y_lrc_pred, average='micro')
lrc_pre=precision_score(y_test, y_lrc_pred, average='micro')

y_lrc_pred2=lrc2.predict(x_test2)
lrc_acc2=accuracy_score(y_test2, y_lrc_pred2)
lrc_rec2=recall_score(y_test, y_lrc_pred2, average='micro')
lrc_pre2=precision_score(y_test, y_lrc_pred2, average='micro')

print("1차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(lrc_acc, lrc_rec, lrc_pre))
print("2차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(lrc_acc2, lrc_rec2, lrc_pre2))

lrc_kscore=(cross_val_score(lrc, data, target, cv=5))
lrc_kscore2=(cross_val_score(lrc2, data, target, cv=5))
print("K-Fold cross validation score: %.4f, %.4f"%(lrc_kscore.mean(), lrc_kscore2.mean()))

Logistic Regression
1차 데이터 예측 정확도: 0.3500, 0.3500, 0.3500
2차 데이터 예측 정확도: 0.4000, 0.4500, 0.4500
K-Fold cross validation score: 0.2000, 0.2000


In [60]:
#Random Forest 모델
print("Random Forest")

forest=RandomForestClassifier(n_estimators=100)
forest.fit(x_train, y_train)

y_for_pred=forest.predict(x_test)
for_acc=accuracy_score(y_test, y_for_pred) 
for_rec=recall_score(y_test, y_for_pred, average='micro')
for_pre=precision_score(y_test, y_for_pred, average='micro')

forest2=RandomForestClassifier(n_estimators=100)
forest2.fit(x_train2, y_train2)

y_for_pred2=forest2.predict(x_test2)
for_acc2=accuracy_score(y_test2, y_for_pred2) 
for_rec2=recall_score(y_test2, y_for_pred2, average='micro')
for_pre2=precision_score(y_test2, y_for_pred2, average='micro')

print("1차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(for_acc, for_rec, for_pre))
print("2차 데이터 예측 정확도: %.4f, %.4f, %.4f"%(for_acc2, for_rec2, for_pre2))

for_kscore=(cross_val_score(forest, data, target, cv=5))
for_kscore2=(cross_val_score(forest2, data, target, cv=5))
print("K-Fold cross validation score: %.4f, %.4f"%(for_kscore.mean(), for_kscore2.mean()))

Random Forest
1차 데이터 예측 정확도: 1.0000, 1.0000, 1.0000
2차 데이터 예측 정확도: 1.0000, 1.0000, 1.0000
K-Fold cross validation score: 0.9900, 0.9700
